In [2]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# Define model function
def build_model(hp):
    model = keras.Sequential()
    # Change input_shape to (784,) to match the reshaped data
    model.add(layers.Dense(hp.Int('units_1', min_value=32, max_value=256, step=32), activation='relu', input_shape=(784,)))

    # Tune number of layers (1 to 3)
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(hp.Int(f'units_{i+2}', min_value=32, max_value=256, step=32), activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    # Tune learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [0.001, 0.01, 0.0001])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [8]:
# Set up tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of models to test
    directory='keras_tuning',
    project_name='hyperparam_tuning'
)

# Load data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
# Reshape data for the model - MNIST images are 28x28 pixels, flatten them
X_train = X_train.reshape(-1, 28*28) # Changed this line
X_test = X_test.reshape(-1, 28*28)  # Changed this line
# Normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
# Convert labels to binary classification (digit 5 or not digit 5)
y_train = (y_train == 5).astype(int) # Changed this line
y_test = (y_test == 5).astype(int)   # Changed this line


# Run search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get best model
best_hyperparameters = tuner.get_best_hyperparameters()[0]
best_model = tuner.get_best_models()[0]

best_model.summary()


Trial 5 Complete [00h 02m 50s]
val_accuracy: 0.9957000017166138

Best val_accuracy So Far: 0.9957000017166138
Total elapsed time: 00h 08m 25s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │         200,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 160)                 │          41,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 283,553 (1.08 MB)

 Trainable params: 283,553 (1.08 MB)

 Non-trainable params: 0 (0.00 B)